# Disney Dataset Creation

Webscraping solution using beautifulsoup
Following along Keith Galli's video: https://www.youtube.com/watch?v=Ewgy-G9cmbg

In [3]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json

In [4]:
# getting the page
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# creating the soup
soup = bs(r.content)

# making it readable
contents = soup.prettify()

In [5]:
# getting only the infobox with the main info
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")


In [6]:
movie_info = {}


def get_content(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find('th').get_text()
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content(row.find('td'))
        movie_info[content_key] = content_value
            

In [7]:
# getting the page
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# creating the soup
soup = bs(r.content)

# making it readable
contents = soup.prettify()

In [25]:
def get_content(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")
    
def clean_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose()
    for tag in soup.find_all("span"):
        tag.decompose()
        
def get_info_box(url):
    r = requests.get(url)
    soup = bs(r.content)
    
    clean_tags(soup)
    
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find('th').get_text()
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content(row.find('td'))
                movie_info[content_key] = content_value
                
    return movie_info

In [26]:
get_info_box('https://en.wikipedia.org/wiki/The_Omega_Connection')


AttributeError: 'NoneType' object has no attribute 'find'

In [27]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select('.wikitable.sortable i a')
base_path = 'https://en.wikipedia.org'
movie_info_list = []

for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
        
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

0
10
20
30
40
Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
50
60
70
80
90
100
110
120
130
140
The London Connection
'NoneType' object has no attribute 'find'
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'find'
490
500
61
'NoneType' object has no attribute 'find_all'
All Night Long
'NoneType' object has no attribute 'find'
510
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
Sister Act 3
'NoneType' object has no attribute 'find'
520
The Thief
'NoneType' object has no attribute 'find_all'
Tom Sawyer
'NoneType' object has no attribute 'find_all'
Tower of Terror
'NoneType' object has no attribute 'find_all'
530
Tron: Ares
'NoneType' object has no attribute 'find'
FC Barc

In [29]:
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [30]:
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [50]:
save_data("disney_data_cleaned.json", movie_info_list)

### Data Cleaning 

In [51]:
movie_info_list = load_data("disney_data_cleaned.json")

#### List of subtasks

1. using python datetime 
2. ~~convert running type and money to integer
3. ~~remove references [1]
4. standardize data
5. ~~some 'starring' are not in a list
6. look at what is going on at the error ones 


In [ ]:
# Clean up references (remove [1], [2])

In [46]:
# convert running time into an integer
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else:
        return int(running_time.split(" ")[0])
        
for movie in movie_info_list:
    movie['Runnig time (int)'] = minutes_to_integer(movie.get('Running time', "N/A"))



In [53]:
print ([movie.get('Budget', 'N/A') for movie in movie_info_list])


['N/A', '$1.49 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', 'N/A', '$788,000', 'N/A', '$1.35 million', '$2.125 million', 'N/A', '$1.5 million', '$1.5 million', 'N/A', '$2.2 million', '$1,800,000', '$3 million', 'N/A', '$4 million', '$2 million', '$300,000', '$1.8 million', 'N/A', '$5 million', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$700,000', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$6 million', 'under $1 million or $1,250,000', 'N/A', '$2 million', 'N/A', 'N/A', '$2.5 million', 'N/A', 'N/A', '$4 million', '$3.6 million', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$3 million', 'N/A', 'N/A', 'N/A', 'N/A', '$4.4–6 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', '$5 million', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', '$4 million', 'N/A', 'N/A', 'N/A', '

In [73]:
# clean up budget & Box office

def dolar_to_integer(budget):
#    if budget.split(' ')[1] == "million":
#        return int(float(budget.split('$')[1].split(" ")[0]) * 1000000)
#    elif budget.split(',')[1] == "000":
        return int(budget.split('$')[1].split(",")[0]) * 1000

dolar_to_integer('$600,000')


600000

In [74]:
x = '$600,000'
print(x.split("$"))

['', '600,000']


Weird
